In [ ]:
!pip install --q "snowflake-connector-python[pandas]"
!sudo pip install --q  snowflake-ml-python==1.0.11 -U
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip install --q seaborn
!pip install --upgrade --q xgboost==1.7.3
!pip install --upgrade --q numpy==1.24.3
!pip install --upgrade --q pandas==1.5.3
!pip install --upgrade --q anyio==3.5.0
!pip install --upgrade --q packaging==23.1
!pip install --upgrade --q scikit-learn==1.3.0
!pip install --upgrade --q typing-extensions==4.7.1
!pip install --upgrade --q cryptography==39.0.0
!pip install --upgrade --q fsspec==2023.9.2
!pip install --q xgboost
!pip install --q catboost
!pip install --q refractml

^C
ERROR: Operation cancelled by user
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier


from xgboost import XGBClassifier
from catboost import CatBoostClassifier

Matplotlib created a temporary cache directory at /tmp/matplotlib-fuofdask because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [3]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [4]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [5]:
df = session.table("AUTO_INSURANCE_CLAIMS_DATA_PRODUCT")
insurance_claim = df.to_pandas()

In [6]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [7]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [8]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [9]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [10]:
df = insurance_claim.copy()

In [11]:
# dropping columns which are not necessary for prediction

to_drop = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']

df.drop(to_drop, inplace=True, axis=1)

In [12]:
# separating the feature and target columns

X = df.drop('fraud_reported', axis = 1)
y = df['fraud_reported']

In [13]:
# extracting categorical columns
cat_df = X.select_dtypes(include = ['object'])

In [14]:
cat_df.columns

Index(['customer_age', 'policy_csl', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_time_of_day', 'property_damage', 'police_report_available'],
      dtype='object')

In [15]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

customer_age: 
['58' '31' '54' '61' '41' '32' '48' '24' '66' '57' '62' '56' '68' '26'
 '28' '64' '43' '46' '42' '21' '49' '59' '52' '47' '22' '63' '65' '69'
 '35' '37' '60' '30' '40' '20' '39' '45' '19' '67' '33' '29' '44' '34'
 '23' '53' '38' '27' '50' '55' '36' '25' '51']

policy_csl: 
['250/500' '100/300' '500/1000']

insured_sex: 
['FEMALE' 'MALE']

insured_education_level: 
['MD' 'College' 'Masters' 'Associate' 'High School' 'PhD' 'JD']

insured_occupation: 
['tech-support' 'adm-clerical' 'sales' 'machine-op-inspct'
 'prof-specialty' 'farming-fishing' 'exec-managerial' 'priv-house-serv'
 'handlers-cleaners' 'other-service' 'armed-forces' 'craft-repair'
 'transport-moving' 'protective-serv']

insured_relationship: 
['other-relative' 'husband' 'wife' 'own-child' 'not-in-family' 'unmarried']

incident_type: 
['Multi-vehicle Collision' 'Single Vehicle Collision' 'Parked Car'
 'Vehicle Theft']

collision_type: 
['Details not Available' 'Front Collision' 'Rear Collision'
 'Side Collisio

In [16]:
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [17]:
num_df = df.select_dtypes(include = ['int16','int8','int32','float64'])

In [18]:
num_df.columns

Index(['months_as_customer', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'capital_gains', 'capital_loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim'],
      dtype='object')

In [19]:
# combining the Numerical and Categorical dataframes to get the final dataset

X = pd.concat([num_df, cat_df], axis = 1)

In [20]:
# splitting data into training set and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [21]:
num_df = X_train[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [22]:
num_df_test = X_test[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [23]:
# Scaling the numeric values in the dataset

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(num_df)

In [24]:
scaled_data_test = scaler.transform(num_df_test)

In [25]:
scaled_num_df = pd.DataFrame(data = scaled_data, columns = num_df.columns, index = X_train.index)
scaled_num_df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
227851,-0.074015,1.509635,0.944327,-0.038324,-0.542885,-0.008837,-0.678438,1.225009,-0.447918,0.808158,-0.980915,1.099058
16455,0.013966,-0.894916,-0.174544,1.303799,-1.182794,-1.033236,-0.678438,1.225009,0.448980,1.944948,-0.592532,0.175680
160610,0.156934,-0.378854,0.436487,-1.560858,-0.523430,1.186295,-0.678438,0.003227,-0.447918,0.311380,-0.162477,-1.230429
97651,-0.524915,0.088594,0.570497,1.709654,-0.335269,0.674095,-0.678438,-1.218555,1.345877,-1.021606,0.200878,-0.115323
203050,0.409878,0.097943,1.223808,0.781013,0.415781,-2.569833,-0.678438,0.003227,-0.447918,1.949279,-0.975706,-0.447642


In [26]:
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X_test.index)
scaled_num_df_test.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
60108,1.355669,-1.065067,1.054285,0.887260,-1.534116,-1.033236,0.747760,1.225009,-1.344815,1.443977,-0.961731,1.590332
126304,-0.271971,-1.538124,-0.695601,-1.096628,-1.112496,0.503362,-0.678438,1.225009,-1.344815,-0.495203,-0.988411,1.312847
227854,1.410656,0.875776,-0.960052,-1.092851,-1.367763,0.503362,-0.678438,-1.218555,1.345877,-1.013082,-0.760234,1.106709
8517,-0.238978,-0.156348,-0.599682,0.646998,-1.179559,0.674095,-0.678438,1.225009,1.345877,-0.589388,-0.978755,-0.207903
110335,0.233917,-0.674281,0.463903,0.992465,-0.081389,-0.179570,0.747760,0.003227,0.448980,0.540275,1.396393,-1.564916


In [27]:
X_train.drop(columns = scaled_num_df.columns, inplace = True)

In [28]:
X_train = pd.concat([scaled_num_df, X_train], axis = 1)

In [29]:
X_test.drop(columns = scaled_num_df_test.columns, inplace = True)

In [30]:
X_test = pd.concat([scaled_num_df_test, X_test], axis = 1)

# Extra Tree Classifier

In [31]:
etc = ExtraTreesClassifier()
etc.fit(X_train, y_train)

ExtraTreesClassifier()

In [32]:
y_pred = etc.predict(X_test)

In [33]:
y_prob = etc.predict_proba(X_test)[:,1]

In [34]:
etc_train_acc = accuracy_score(y_train, etc.predict(X_train))
etc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Extra Tree is : {etc_train_acc}")
print(f"Test accuracy of Extra Tree is : {etc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Extra Tree is : 1.0
Test accuracy of Extra Tree is : 0.9968412245803951
[[ 8419   181]
 [    4 49963]]
                   precision    recall  f1-score   support

   Fraud Reported       1.00      0.98      0.99      8600
No Fraud Reported       1.00      1.00      1.00     49967

         accuracy                           1.00     58567
        macro avg       1.00      0.99      0.99     58567
     weighted avg       1.00      1.00      1.00     58567



In [35]:
from refractml import *
from refractml.constants import MLModelFlavours
import requests

In [36]:
actual_model_columns = ['months_as_customer',
 'policy_deductable',
 'umbrella_limit',
 'capital_gains',
 'capital_loss',
 'incident_hour_of_the_day',
 'number_of_vehicles_involved',
 'bodily_injuries',
 'witnesses',
 'injury_claim',
 'property_claim',
 'vehicle_claim',
 'policy_annual_premium',
 'total_claim_amount',
 'customer_age_20',
 'customer_age_21',
 'customer_age_22',
 'customer_age_23',
 'customer_age_24',
 'customer_age_25',
 'customer_age_26',
 'customer_age_27',
 'customer_age_28',
 'customer_age_29',
 'customer_age_30',
 'customer_age_31',
 'customer_age_32',
 'customer_age_33',
 'customer_age_34',
 'customer_age_35',
 'customer_age_36',
 'customer_age_37',
 'customer_age_38',
 'customer_age_39',
 'customer_age_40',
 'customer_age_41',
 'customer_age_42',
 'customer_age_43',
 'customer_age_44',
 'customer_age_45',
 'customer_age_46',
 'customer_age_47',
 'customer_age_48',
 'customer_age_49',
 'customer_age_50',
 'customer_age_51',
 'customer_age_52',
 'customer_age_53',
 'customer_age_54',
 'customer_age_55',
 'customer_age_56',
 'customer_age_57',
 'customer_age_58',
 'customer_age_59',
 'customer_age_60',
 'customer_age_61',
 'customer_age_62',
 'customer_age_63',
 'customer_age_64',
 'customer_age_65',
 'customer_age_66',
 'customer_age_67',
 'customer_age_68',
 'customer_age_69',
 'policy_csl_250/500',
 'policy_csl_500/1000',
 'insured_sex_MALE',
 'insured_education_level_College',
 'insured_education_level_High School',
 'insured_education_level_JD',
 'insured_education_level_MD',
 'insured_education_level_Masters',
 'insured_education_level_PhD',
 'insured_occupation_armed-forces',
 'insured_occupation_craft-repair',
 'insured_occupation_exec-managerial',
 'insured_occupation_farming-fishing',
 'insured_occupation_handlers-cleaners',
 'insured_occupation_machine-op-inspct',
 'insured_occupation_other-service',
 'insured_occupation_priv-house-serv',
 'insured_occupation_prof-specialty',
 'insured_occupation_protective-serv',
 'insured_occupation_sales',
 'insured_occupation_tech-support',
 'insured_occupation_transport-moving',
 'insured_relationship_not-in-family',
 'insured_relationship_other-relative',
 'insured_relationship_own-child',
 'insured_relationship_unmarried',
 'insured_relationship_wife',
 'incident_type_Parked Car',
 'incident_type_Single Vehicle Collision',
 'incident_type_Vehicle Theft',
 'collision_type_Front Collision',
 'collision_type_Rear Collision',
 'collision_type_Side Collision',
 'incident_severity_Minor Damage',
 'incident_severity_Total Loss',
 'authorities_contacted_Fire',
 'authorities_contacted_None',
 'authorities_contacted_Other',
 'authorities_contacted_Police',
 'incident_time_of_day_Early Morning Hours',
 'incident_time_of_day_Morning to Noon',
 'incident_time_of_day_Night Time',
 'property_damage_No Property Damage',
 'property_damage_Property Damage',
 'police_report_available_Police Report Available',
 'police_report_available_Unknown']

In [37]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    df = pd.DataFrame(payload_dict,index=[0])
    df = df.fillna(0)
    df.columns = df.columns.str.lower()
    df.drop(to_drop, inplace=True, axis=1)
    
    # extracting categorical columns
    cat_df = df[['customer_age','policy_csl','insured_sex','insured_education_level','insured_occupation','insured_relationship','incident_type','collision_type','incident_severity','authorities_contacted','incident_time_of_day','property_damage','police_report_available']]
    cat_df = pd.get_dummies(cat_df, drop_first = True)

    num_df = df[['months_as_customer', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'capital_gains', 'capital_loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim']]
    
    X = pd.concat([num_df, cat_df], axis = 1)
    
    num_df_test = X[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

    scaled_data_test = scaler.transform(num_df_test)
    scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X.index)
    
    X.drop(columns = scaled_num_df_test.columns, inplace = True)
    X = pd.concat([scaled_num_df_test, X], axis = 1)
    
    missing_features = [missing_col for missing_col in actual_model_columns if missing_col not in X.columns]
    X[missing_features] = 0
    
    prediction = model.predict(X[actual_model_columns])
    probability = model.predict_proba(X)[:,1]
    return {"prediction" : prediction, "probability" : probability}

In [38]:
insurance_claim.columns = insurance_claim.columns.str.upper()
insurance_claims = insurance_claim.copy()
insurance_claims.drop('FRAUD_REPORTED', axis = 1, inplace=True)
payload = insurance_claims.iloc[0].to_dict()

In [40]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
y_out = score(etc, y_req)
y_out

{'prediction': array(['No Fraud Reported'], dtype=object),
 'probability': array([1.])}

In [41]:
req.json

{'payload': {'MONTHS_AS_CUSTOMER': 37,
  'CUSTOMER_AGE': '58',
  'POLICY_NUMBER': 459791,
  'POLICY_BIND_DATE': '27-04-2010',
  'POLICY_STATE': 'CT',
  'POLICY_CSL': '250/500',
  'POLICY_DEDUCTABLE': 1171,
  'POLICY_ANNUAL_PREMIUM': 1394,
  'UMBRELLA_LIMIT': 4344805,
  'INSURED_ZIP': '444558',
  'INSURED_SEX': 'FEMALE',
  'INSURED_EDUCATION_LEVEL': 'MD',
  'INSURED_OCCUPATION': 'tech-support',
  'INSURED_HOBBIES': 'board-games',
  'INSURED_RELATIONSHIP': 'other-relative',
  'CAPITAL_GAINS': 77256,
  'CAPITAL_LOSS': -9989,
  'INCIDENT_DATE': datetime.date(2022, 5, 13),
  'INCIDENT_TYPE': 'Multi-vehicle Collision',
  'COLLISION_TYPE': 'Details not Available',
  'INCIDENT_SEVERITY': 'Major Damage',
  'AUTHORITIES_CONTACTED': 'None',
  'INCIDENT_STATE': 'CT',
  'INCIDENT_CITY': 'Vernon',
  'INCIDENT_LOCATION': 'Lane',
  'INCIDENT_HOUR_OF_THE_DAY': 22,
  'INCIDENT_TIME_OF_DAY': 'Night Time',
  'NUMBER_OF_VEHICLES_INVOLVED': 3,
  'PROPERTY_DAMAGE': 'No Property Damage',
  'BODILY_INJURIES': 

In [42]:
## registering the model in refract.
model_reg = register_model(etc, 
               score, 
               name="ILF_ExtraTree_FraudClassifier", 
               description="Insurance claim's model trained for Fraud Classification",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_pred, 
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
